## Merge all csv files

In [5]:
import pandas as pd
import os

def merge_csv_files(directory_path, output_file):
    # List to hold data from each CSV file
    dataframes = []

    # Loop through all files in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith('.csv'):
            # Construct full file path
            file_path = os.path.join(directory_path, filename)
            # Read the CSV file and append to the list
            print(file_path)
            df = pd.read_csv(file_path)
            dataframes.append(df)

    # Concatenate all dataframes in the list
    merged_df = pd.concat(dataframes, ignore_index=True)

    # Write the merged dataframe to a new CSV file
    merged_df.to_csv(output_file, index=False)
    print(f"All CSV files have been merged into {output_file}")

# Usage
raw_data_path = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/raw_data'
merged_data_path = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/merged_data.csv'
merge_csv_files(raw_data_path, merged_data_path)

/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/raw_data/etf_holdings_600_stocks_2010_01_2013_12.csv
/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/raw_data/etf_holdings_600_stocks_2014_01_2015_11.csv
/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/raw_data/test.csv
/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/raw_data/etf_holdings_600_stocks_2020_01_2020_12.csv
/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/raw_data/etf_holdings_600_stocks_2021_01_2022_12.csv
/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/raw_data/etf_holdings_600_stocks_2019_01_2019_12.csv
/Users/jonathan

# Separate ETFs and traditional Funds

## filter_function
This function loads merged_data.csv and assigns funds to one of the following categories: `ETF`, `index_fund`, `active_fund`. Further the function allows for a filter on the location of the fund.

In [3]:
import dask.dataframe as dd
import pandas as pd

def country_filter(filtered_ddf):
    selected_countries = ["Albania", "Andorra", "Armenia", "Austria", "Azerbaijan", "Belarus", "Belgium", "Bosnia and Herzegovina",
    "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Georgia", 
    "Germany", "Greece", "Hungary", "Iceland", "Ireland", "Italy", "Kazakhstan", "Kosovo", "Latvia", "Liechtenstein", 
    "Lithuania", "Luxembourg", "Malta", "Moldova", "Monaco", "Montenegro", "Netherlands", "North Macedonia", 
    "Norway", "Poland", "Portugal", "Romania", "San Marino", "Serbia", "Slovakia", "Slovenia", 
    "Spain", "Sweden", "Switzerland", "Turkey", "Ukraine", "United Kingdom", "Vatican City"]

    return filtered_ddf[filtered_ddf['country'].isin(selected_countries)]


def filter_function(input_file, output_file, type = "etf", subset = "none"):
    ddf = dd.read_csv(input_file)
    # Drop rows with NaN in 'fund_type' or 'fund_name'
    ddf = ddf.dropna(subset=["stock_RIC", 'fund_type', 'fund_name', "percent_of_traded_shares"])
    
    ########
    if type == "etf":
        filtered_ddf = ddf[(ddf['fund_type'] == 'Exchange-Traded Fund')]

    elif type == "mutual fund":
        filtered_ddf = ddf[(ddf['fund_type'] != 'Exchange-Traded Fund')]
    
    ######## type = "index fund" or type = "active fund" needs mutual fund data as input_file
    elif type == "index fund":
        filtered_ddf = ddf[(ddf['fund_investment_type'] == 'Index')]
    
    elif type == "active fund":
        filtered_ddf = ddf[(ddf['fund_investment_type'] != 'Index')]

    ######## call country_filter
    if subset == "europe":
        filtered_ddf = country_filter(filtered_ddf)
    
    filtered_ddf = filtered_ddf.persist()
    filtered_df = filtered_ddf.compute()

    if type == "etf" or type == "mutual fund":
        filtered_df['date'] = pd.to_datetime(filtered_df['date'])

        # Subtract one month end to change the date to the last day of the previous month
        filtered_df['date'] = filtered_df['date'] - pd.offsets.MonthEnd(1)

    # Identify and drop unnamed columns
    df_clean = filtered_df.loc[:, ~filtered_df.columns.str.contains('Unnamed')]
    df_clean = df_clean.drop_duplicates(subset=['stock_RIC', 'fund_name', 'date', 'percent_of_traded_shares'], keep='first')

    # Compute and write the result to a new CSV file
    df_clean.to_csv(output_file)

## ETFs

In [4]:
import dask.dataframe as dd
import pandas as pd
    
########## ETFs
input_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/merged_data.csv'
output_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/etf_data.csv'
output_file_europe = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/etf_data_europe.csv'

filter_function(input_file, output_file, type = "etf")
filter_function(input_file, output_file_europe, type = "etf", subset = "europe")

## Mutual Funds

In [5]:
import dask.dataframe as dd
import pandas as pd

########## traditional funds
input_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/merged_data.csv'
output_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/fund_data.csv'
output_file_europe = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/fund_data_europe.csv'

filter_function(input_file, output_file, type = "mutual fund")
filter_function(input_file, output_file_europe, type = "mutual fund", subset = "europe")

## Index Funds

In [6]:
import dask.dataframe as dd
import pandas as pd

########## index funds
input_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/fund_data.csv'
output_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/index_fund_data.csv'
output_file_europe = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/index_fund_data_europe.csv'

#filter_function(input_file, output_file, type = "index fund")
filter_function(input_file, output_file_europe, type = "index fund", subset = "europe")

## Active Funds

In [7]:
import dask.dataframe as dd
import pandas as pd

########## active funds
input_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/fund_data.csv'
output_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/active_fund_data.csv'
output_file_europe = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/active_fund_data_europe.csv'

#filter_function(input_file, output_file, type = "active fund")
filter_function(input_file, output_file_europe, type = "active fund", subset = "europe")

# Aggregation of Ownership

## aggregation_function

In [8]:
import pandas as pd
def aggregation_function(input_file, output_file, type = "etf"):
    df = pd.read_csv(input_file, index_col=False)
    
    grouped_df = df.groupby(['stock_RIC', 'date'])[['stock_value_held', "percent_of_traded_shares"]].sum().reset_index()

    if type == "mutual fund":
        grouped_df.rename(columns={'stock_value_held': 'FUND_stock_value_held'},  inplace=True)
        grouped_df.rename(columns={'percent_of_traded_shares': 'FUND_percent_of_traded_shares'}, inplace=True)
    
    elif type == "index fund":
        grouped_df.rename(columns={'stock_value_held': 'INDEX_FUND_stock_value_held'},  inplace=True)
        grouped_df.rename(columns={'percent_of_traded_shares': 'INDEX_FUND_percent_of_traded_shares'}, inplace=True)
    
    elif type == "active fund":
        grouped_df.rename(columns={'stock_value_held': 'ACTIVE_FUND_stock_value_held'},  inplace=True)
        grouped_df.rename(columns={'percent_of_traded_shares': 'ACTIVE_FUND_percent_of_traded_shares'}, inplace=True)

    grouped_df.to_csv(output_file, index=False)

## ETF ownership

In [9]:
#### without any filter
input_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/etf_data.csv'
output_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/aggregated_data/etf_aggregated_data.csv'
aggregation_function(input_file, output_file, type = "etf")

#### europe filter
input_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/etf_data_europe.csv'
output_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/aggregated_data/etf_aggregated_data_europe.csv'
aggregation_function(input_file, output_file, type = "etf")

## Mutual Fund ownership

In [10]:
#### without any filter
input_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/fund_data.csv'
output_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/aggregated_data/fund_aggregated_data.csv'
aggregation_function(input_file, output_file, type = "mutual fund")

#### europe filter
input_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/fund_data_europe.csv'
output_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/aggregated_data/fund_aggregated_data_europe.csv'
aggregation_function(input_file, output_file, type = "mutual fund")

## Index Fund ownership

In [11]:
#### without any filter
input_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/index_fund_data.csv'
output_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/aggregated_data/index_fund_aggregated_data.csv'
aggregation_function(input_file, output_file, type = "index fund")

#### europe filter
input_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/index_fund_data_europe.csv'
output_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/aggregated_data/index_fund_aggregated_data_europe.csv'
aggregation_function(input_file, output_file, type = "index fund")

## Active Fund ownership

In [12]:
#### without any filter
input_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/active_fund_data.csv'
output_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/aggregated_data/active_fund_aggregated_data.csv'
aggregation_function(input_file, output_file, type = "active fund")

#### europe filter
input_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/fund_type_filtered/active_fund_data_europe.csv'
output_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/aggregated_data/active_fund_aggregated_data_europe.csv'
aggregation_function(input_file, output_file, type = "active fund")

# Merge aggregated ETF and Fund ownership

In [13]:
import pandas as pd

def merge_csv_files(file1, file2, output_file, column_added):
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    key_columns = ['date', 'stock_RIC']

    df1['date'] = pd.to_datetime(df1['date'])
    df2['date'] = pd.to_datetime(df2['date'])

    cols_to_keep = key_columns + column_added
    df2 = df2[cols_to_keep]

    merged_df = pd.merge(df1, df2, on=key_columns, how='left')

    # Fill NA values in 'stock_value_held' with the desired placeholder if no match was found
    merged_df[column_added].fillna('NA', inplace=True)

    # Write the merged DataFrame to a new CSV file
    merged_df.to_csv(output_file, index=False)
    print(f"Merged data has been saved to {output_file}")

def apply_merge_function(subset = "none"):
    formatted_index_member = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/index_constituents_data/formated_constituents.csv'
    file_m_stock = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/m_stock_level_data.csv'
    
    if subset == "europe":
        file_etf = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/aggregated_data/etf_aggregated_data_europe.csv'
        file_mutual_fund = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/aggregated_data/fund_aggregated_data_europe.csv"
        file_index_fund = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/aggregated_data/index_fund_aggregated_data_europe.csv"
        file_active_fund = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/aggregated_data/active_fund_aggregated_data_europe.csv"
        output_file = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/formatted_final_europe.csv"
    else:
        file_etf = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/aggregated_data/etf_aggregated_data.csv'
        file_mutual_fund = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/aggregated_data/fund_aggregated_data.csv"
        file_index_fund = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/aggregated_data/index_fund_aggregated_data.csv"
        file_active_fund = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/aggregated_data/active_fund_aggregated_data.csv"
        output_file = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/formatted_final.csv"

    ##### merge etf ownership with index memeber
    merge_csv_files(formatted_index_member, file_etf, output_file, ['stock_value_held', "percent_of_traded_shares"])

    ##### merge fund ownership with the output file from the last function
    merge_csv_files(output_file, file_mutual_fund, output_file, ["FUND_stock_value_held", "FUND_percent_of_traded_shares"])
    merge_csv_files(output_file, file_index_fund, output_file, ["INDEX_FUND_stock_value_held", "INDEX_FUND_percent_of_traded_shares"])
    merge_csv_files(output_file, file_active_fund, output_file, ["ACTIVE_FUND_stock_value_held", "ACTIVE_FUND_percent_of_traded_shares"])

    #### add stock level data
    merge_csv_files(output_file, file_m_stock, output_file, ['market_cap'])

apply_merge_function(subset = "none")
apply_merge_function(subset = "europe")

/var/folders/m9/n85vv59j4lj5g5nndc07m8t40000gn/T/ipykernel_62090/372549136.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[column_added].fillna('NA', inplace=True)


Merged data has been saved to /Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/formatted_final_europe.csv


/var/folders/m9/n85vv59j4lj5g5nndc07m8t40000gn/T/ipykernel_62090/372549136.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[column_added].fillna('NA', inplace=True)


Merged data has been saved to /Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/formatted_final_europe.csv


/var/folders/m9/n85vv59j4lj5g5nndc07m8t40000gn/T/ipykernel_62090/372549136.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[column_added].fillna('NA', inplace=True)


Merged data has been saved to /Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/formatted_final_europe.csv


/var/folders/m9/n85vv59j4lj5g5nndc07m8t40000gn/T/ipykernel_62090/372549136.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[column_added].fillna('NA', inplace=True)


Merged data has been saved to /Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/formatted_final_europe.csv


/var/folders/m9/n85vv59j4lj5g5nndc07m8t40000gn/T/ipykernel_62090/372549136.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[column_added].fillna('NA', inplace=True)


Merged data has been saved to /Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/formatted_final_europe.csv


## calculate ownership percentage based on market capitalization

In [14]:
import pandas as pd

def calculate_ownership_percentage(subset = "none"):
    output_file = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/formatted_final.csv"
    if subset == "europe":
        output_file = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/formatted_final_europe.csv"
    
    df = pd.read_csv(output_file, index_col=False)
    df['ETF_ownership'] = (df["stock_value_held"] * 1_000_000)/ df["market_cap"]
    df['FUND_ownership'] = (df["FUND_stock_value_held"] * 1_000_000)/ df["market_cap"]
    df['INDEX_FUND_ownership'] = (df["INDEX_FUND_stock_value_held"] * 1_000_000)/ df["market_cap"]
    df['ACTIVE_FUND_ownership'] = (df["ACTIVE_FUND_stock_value_held"] * 1_000_000)/ df["market_cap"]
    df.to_csv(output_file, index=False)
    display(df)

calculate_ownership_percentage(subset = "none")
calculate_ownership_percentage(subset = "europe")

,date,stock_RIC,index_member_600,index_member_50,stock_value_held,percent_of_traded_shares,FUND_stock_value_held,FUND_percent_of_traded_shares,INDEX_FUND_stock_value_held,INDEX_FUND_percent_of_traded_shares,ACTIVE_FUND_stock_value_held,ACTIVE_FUND_percent_of_traded_shares,market_cap,ETF_ownership,FUND_ownership,INDEX_FUND_ownership,ACTIVE_FUND_ownership
0,2009-12-31,0MW4EUR.xbo^K15,0,0,3.073419,0.1655,101.889071,4.7543,3.486577,0.1713,98.402494,4.5830,NaN,NaN,NaN,NaN,NaN
1,2010-01-31,0MW4EUR.xbo^K15,0,0,3.036434,0.1629,98.684809,4.5919,3.486577,0.1713,95.198232,4.4206,NaN,NaN,NaN,NaN,NaN
2,2010-02-28,0MW4EUR.xbo^K15,0,0,3.036434,0.1629,97.881422,4.5459,3.184193,0.1540,94.697229,4.3919,NaN,NaN,NaN,NaN,NaN
3,2010-03-31,0MW4EUR.xbo^K15,0,0,2.824438,0.1533,106.506665,4.8721,3.270862,0.1546,103.235803,4.7175,NaN,NaN,NaN,NaN,NaN
4,2010-04-30,0MW4EUR.xbo^K15,0,0,2.850560,0.1539,96.433853,4.3504,5.939661,0.2685,90.494192,4.0819,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196219,2023-07-31,ZURN.S,1,1,2052.542650,3.1067,11438.503679,17.3140,3199.414024,4.8451,8239.089655,12.4689,6.606391e+10,0.031069,0.173143,0.048429,0.124714
196220,2023-08-31,ZURN.S,1,1,2039.457827,3.2077,11315.144184,17.7587,3195.650139,5.0353,8119.494045,12.7234,6.344198e+10,0.032147,0.178354,0.050371,0.127983
196221,2023-09-30,ZURN.S,1,1,1996.051120,3.1388,11221.412654,17.5905,3117.850540,4.9036,8103.562114,12.6869,6.357877e+10,0.031395,0.176496,0.049039,0.127457
196222,2023-10-31,ZURN.S,1,1,2101.553892,3.2116,11505.237587,17.6709,3250.279289,4.9704,8254.958298,12.7005,6.545329e+10,0.032108,0.175778,0.049658,0.126120
